In [10]:
import os
from collections import defaultdict
from datetime import datetime
from os.path import basename, exists, join

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import pynetdicom
from pydicom.uid import UID
from PIL import Image
from tqdm import trange

%matplotlib inline

In [11]:
def jpg2dcm(img, PatientID,AccessionNumber, StudyInstanceUID, SeriesInstanceUID, SOPInstanceUID, InstanceNumber):
    sc_template_path = 'D:/dazhou/template.dcm'
    ds = pydicom.read_file(sc_template_path)

    # operate file_mata tags
    ds.file_meta.MediaStorageSOPInstanceUID = SOPInstanceUID

    # set creation date/time
    dt = datetime.now()
    ds.StudyDate = dt.strftime("%Y%m%d")
    ds.StudyTime = dt.strftime("%H%M%S.%f")

    # write meta tags
    ds.PatientID = PatientID
    ds.AccessionNumber = AccessionNumber
    ds.StudyID = AccessionNumber
    ds.StudyInstanceUID = StudyInstanceUID
    ds.SeriesInstanceUID = SeriesInstanceUID
    ds.SOPInstanceUID = SOPInstanceUID
    ds.InstanceNumber = InstanceNumber

    ds.PatientName = "anonymous"
    ds.PatientBirthDate = ""
    ds.PatientBirthTime = ""
    # if ds[ds['PatientSex'].tag]:
    #     del ds[ds['PatientSex'].tag]
    # if  ds[ds['PatientAge'].tag]:
    #     del ds[ds['PatientAge'].tag]

#     ds.ImagePositionPatient = r"0\0\{}".format(InstanceNumber)
#     ds.ImageOrientationPatient = r"1\0\0\0\-1\0"
#     ds.PixelSpacing = [1, 1, 1]
    ds.Rows, ds.Columns = img.shape[:2]
    ds.PixelData = img.tobytes()
    
    return ds

In [12]:
data_root = 'D:/dazhou/dazhou_forlabelsys_all_2339_2000_2000_2000'
convert_data_root = 'D:/dazhou/ThyroidDcmData'
pids = os.listdir(data_root)

log_file = open('D:/dazhou/record.log', mode='a+', encoding='utf-8')
total_num = 0
for i in trange(len(pids)):
    pid = pids[i]
    total_num += 1
    files = os.listdir(join(data_root, pid))
    files = sorted(files)
    
    # print(total_num, pid)
    studies = defaultdict(list)
    for f in files:
        if f.startswith('SC.1.2.840.10008.5') and not 'aug' in f.split('.')[-1]:
            key = '.'.join(f.split('.')[:7])
            studies[key].append(f)
        else:
            error_f_path = join(data_root, pid, f)
            log_file.write(error_f_path + '\n')
            log_file.flush()

    for study, series in studies.items():
        cur_SeriesInstanceUID = None
        for sop_idx, sop in enumerate(series):
            sopitems = sop.replace('.jpg', '').split('.')
            newsopitems = sopitems[1:5].copy()
            newsopitems.append(sopitems[5] + sopitems[6])

            img = np.array(Image.open(join(data_root, pid, sop)),
                           dtype=np.uint8)
            PatientID = pid
            AccessionNumber = sopitems[-2]
            StudyInstanceUID = UID(".".join(newsopitems))
            #         SeriesInstanceUID = UID(".".join(sopitems[1:8]))
            # 统一同一个study下的SeriesInstanceUID
            if cur_SeriesInstanceUID:
                SeriesInstanceUID = cur_SeriesInstanceUID
            else:
                cur_SeriesInstanceUID = UID(".".join(sopitems[1:]))
                SeriesInstanceUID = cur_SeriesInstanceUID
            SOPInstanceUID = UID(".".join(sopitems[1:8]))
            InstanceNumber = sop_idx + 1

            #         print("PatientID", PatientID)
            #         print("AccessionNumber", AccessionNumber)
            #         print("StudyInstanceUID", StudyInstanceUID)
            #         print("SeriesInstanceUID", SeriesInstanceUID)
            #         print("SOPInstanceUID", SOPInstanceUID)
            #         print("InstanceNumber", InstanceNumber)

            ds = jpg2dcm(img, PatientID, AccessionNumber, StudyInstanceUID,
                         SeriesInstanceUID, SOPInstanceUID, InstanceNumber)

            new_pdir = join(convert_data_root, pid)
            if not os.path.exists(new_pdir):
                os.makedirs(new_pdir, exist_ok=True)
            new_path = join(new_pdir, "SC.{}.dcm".format(SOPInstanceUID))
            ds.save_as(new_path)

        cur_SeriesInstanceUID = None
    # break
log_file.close()

100%|██████████| 8339/8339 [1:08:47<00:00,  2.02it/s]


In [13]:
"""
    用来查看里面有多少个instance数据
"""
root = "D:/dazhou/dazhou_forlabelsys_all_2339_2000_2000_2000"
patients_paths = os.listdir(root)
cnt = 0
a = set()
for i in trange(len(patients_paths)):
    patients_path = patients_paths[i]
    patients = os.listdir(os.path.join(root,patients_path))
    for p in patients:
        str = '.'.join(p.split('.')[0:-2])
        a.add(str)   
    cnt+= len(patients)

print(len(a))
print(cnt)

100%|██████████| 8339/8339 [01:17<00:00, 106.98it/s] 

17368
109652


In [ ]:
"""clear pacs"""

In [14]:
import psycopg2

ip = '192.168.7.197'
db_port = 15432
db_name = 'pacsdb'
db_user = 'pacs'
db_pwd = 'pacs'


# 创建数据库连接，数据库配置信息根据实际情况填写
LocalPostgreSQLConnection = psycopg2.connect(
    host=ip, port=db_port, database=db_name, user=db_user, password=db_pwd)
cur = LocalPostgreSQLConnection.cursor()


def deleteLocation(instance_fk):
    # 删除location
    sql = "delete from location where instance_fk={}".format(instance_fk)
    cur.execute(sql)


def deleteInstance(series_fk):
    # 查询instance
    sql = "select * from instance where series_fk={}".format(series_fk)
    cur.execute(sql)
    instances = cur.fetchall()
    # 遍历Instance依赖
    for instance in instances:
        # 删除location
        deleteLocation(instance[0])

    # 删除instance
    sql = "delete from instance where series_fk={}".format(series_fk)
    cur.execute(sql)


def deleteSeriesQueryAttrs(series_fk):
    # 删除study_query_attrs
    sql = "delete from series_query_attrs where series_fk={}".format(series_fk)
    cur.execute(sql)


def deleteSeriesReq(series_fk):
    # 删除SeriesReq
    sql = "delete from series_req where series_fk={}".format(series_fk)
    cur.execute(sql)


def deleteSeries(study_fk):
    # 查询series
    sql = "select * from series where study_fk={}".format(study_fk)
    cur.execute(sql)
    serieses = cur.fetchall()
    # 遍历series依赖
    for series in serieses:
        # 删除instance
        deleteInstance(series[0])
        # 删除series_query_attrs
        deleteSeriesQueryAttrs(series[0])
        # 删除SeriesReq
        deleteSeriesReq(series[0])

    # 删除series
    sql = "delete from series where study_fk={}".format(study_fk)
    cur.execute(sql)


def deleteStudyQueryAttrs(study_fk):
    # 删除study_query_attrs
    sql = "delete from study_query_attrs where study_fk={}".format(study_fk)
    cur.execute(sql)


def deleteStudy(patient_fk):
    # 查询study
    sql = "select * from study where patient_fk={}".format(patient_fk)
    cur.execute(sql)
    studys = cur.fetchall()
    # 遍历study依赖
    for study in studys:
        # 删除series
        deleteSeries(study[0])
        # 删除study_query_attrs
        deleteStudyQueryAttrs(study[0])

    # 删除study
    sql = "delete from study where patient_fk={}".format(patient_fk)
    cur.execute(sql)


def deletePatient(patient_id):
    # 获取对应patient_ID
    sql = "select * from patient_id where pat_id='{}'".format(patient_id)
    cur.execute(sql)
    patient_ID = cur.fetchall()[0]
    patient_fk = patient_ID[0]
    # 删除对应study
    deleteStudy(patient_fk)
    # 删除对应patient
    sql = "delete from patient where pk={}".format(patient_fk)
    cur.execute(sql)
    # 删除对应patient_id
    sql = "delete from patient_id where pat_id='{}'".format(patient_id)
    cur.execute(sql)


def clearDatabase():
    # 获取patient_id列表
    sql = "select * from patient_id"
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        deletePatient(row[1])


def commit():
    LocalPostgreSQLConnection.commit()
    LocalPostgreSQLConnection.close()


clearDatabase()
commit()


In [16]:
"""向pacs 发送数据"""
import os
import pydicom
from tqdm import trange

from pydicom import dcmread
from pynetdicom import AE,debug_logger,ALL_TRANSFER_SYNTAXES,AllStoragePresentationContexts
from pydicom.uid import ImplicitVRLittleEndian

In [26]:
def send(patients,root):
    patientDcms = []
    ae = AE(ae_title="DCM4CHEE")
    ae.supported_contexts = AllStoragePresentationContexts
    # print(patients)
    tmp_img_path = os.path.join(root,patients[0])
    tmp = pydicom.read_file(tmp_img_path, force=True)
    ae.add_requested_context(tmp.SOPClassUID)
    for patient in patients:
        img_path = root + "/" + patient
        dcm = pydicom.read_file(img_path, force=True)
        if not dcm.file_meta.TransferSyntaxUID:
            dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
        patientDcms.append(dcm)

    assoc = ae.associate("192.168.7.197", 15112, ae_title="DCM4CHEE")
    success_number = 0
    error_list = []
    if assoc.is_established:
        for ima in patientDcms:
            status = assoc.send_c_store(ima)
            if status:
                success_number += 1
            else:
                error_list.append(ima.StudyInstanceUID)
                print("failed: ", ima.PatientName)

    assoc.release()

In [27]:
root = "D:/dazhou/ThyroidDcmData/"
patients_paths = os.listdir(root)
for i in trange(len(patients_paths)):
    patients_path = patients_paths[i]
    patients = os.listdir(os.path.join(root,patients_path))
    send(patients,os.path.join(root,patients_path))

100%|██████████| 8336/8336 [2:43:57<00:00,  1.18s/it]  
